In [1]:
from sloyka import PostGetter

/home/gk/.cache/pypoetry/virtualenvs/factfinder-6OvYRBkP-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /home/gk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
